**Table of contents**<a id='toc0_'></a>    
- [Robot Arm](#toc1_)    
- [pybulletの起動](#toc2_)    
- [Initial Setup for pybullet](#toc3_)    
- [Generating the Robot Arm](#toc4_)    
- [Defining Functions for Inverse Kinematics Using Analytical Methods](#toc5_)    
- [Running the Simulation](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Robot Arm](#toc0_)

In this notebook, we will generate a 2-axis robot arm and explain how to solve inverse kinematics using analytical methods.

(For a manual summarizing the functions available in pybullet, please refer to [this link](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = Mesa
ven = Mesa


# <a id='toc3_'></a>[Initial Setup for pybullet](#toc0_)

In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Set the camera position for GUI mode
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# <a id='toc4_'></a>[Generating the Robot Arm](#toc0_)
This time, we will generate a 2-axis robot arm `simple_2d_arm.urdf`.  
The robot is configured as shown in the diagram below (sensors are explained in `robot_arm_sensor.ipynb`).  

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# Load the robot
arm_start_pos = [0, 0, 0.1]  # Set initial position (x, y, z)
arm_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set initial orientation (roll, pitch, yaw)
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf", arm_start_pos, arm_start_orientation, useFixedBase=True) # Fix the root link to prevent the robot from falling over by setting useFixedBase=True

# Set the camera position for GUI mode
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[Defining Functions for Inverse Kinematics Using Analytical Methods](#toc0_)
We will define functions for inverse kinematics using analytical methods.

<br>

There are two methods for solving inverse kinematics analytically:
- Algebraic method
- Geometric method

In [4]:
import math
def inverse_kinematics_2link(link1_length, link2_length, xe, ye):
    """
    Calculate the inverse kinematics of a 2-link robot arm in a 2D plane
    
    Parameters
    ----------
    link1_length : float
        Length of link 1
    link2_length : float
        Length of link 2
    xe : float
        x-coordinate of the end effector
    ye : float
        y-coordinate of the end effector

    Returns
    -------
    theta1 : float
        Joint angle of link 1 (rad)
    theta2 : float
        Joint angle of link 2 (rad)
    """
    try:
        # Inverse kinematics equations derived algebraically
        theta1 = -math.acos((xe**2 + ye**2 + link1_length**2 - link2_length**2)/(2 * link1_length * math.sqrt(xe**2 + ye**2))) + math.atan2(ye, xe)
        theta2 = math.acos((xe**2 + ye**2 - link1_length**2 - link2_length**2)/(2 * link1_length * link2_length))

        # Inverse kinematics equations derived geometrically (uncomment the following lines if you want to try the "geometric" method)
        # theta1 = math.atan2(ye, xe) - math.acos((-l2**2 + l1**2 + xe**2 + ye**2)/(2 * l1 * math.sqrt(xe**2 + ye**2)))
        # theta2 = math.pi - math.acos((-xe**2 - ye**2 + l2**2 + l1**2)/(2 * l2 * l1))
    except: # If the input (xe, ye) does not have a solution, output None
        theta1 = None
        theta2 = None
    return theta1, theta2

# <a id='toc6_'></a>[Running the Simulation](#toc0_)

When the simulation is run, the end-effector of the robot arm will be set to the "specified end-effector position `(xe, ye)`".


In [21]:
# Length of the robot links
LINK1_LENGTH = 0.5   # Length of link1 (z-direction length of link1 in "simple2d_arm.urdf")
LINK2_LENGTH = 0.55  # Length of link2 (z-direction length of link2 + force_sensor_link in "simple2d_arm.urdf")

# Index of each joint of the robot
LINK1_JOINT_IDX = 0
LINK2_JOINT_IDX = 1

##### Changing this will change the result (please specify a feasible end-effector position) #####
# Coordinates of the end effector
xe = 0.0
ye = 0.8
############################################

# Calculate inverse kinematics using analytical methods
theta1, theta2 = inverse_kinematics_2link(LINK1_LENGTH, LINK2_LENGTH, xe, ye)

# Set the joint angles obtained by inverse kinematics to the robot (confirm that the specified end-effector (xe, ye) is set)
if theta1 is not None and theta2 is not None:
    pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, theta1)
    pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, theta2)
    pybullet.stepSimulation()
else:
    print("No solution exists for the inverse kinematics")